In [56]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
from multielo import MultiElo, Player, Tracker
from mktools.get_data import load_data_pd
from alive_progress import alive_it

load_dotenv()

True

In [58]:
df = load_data_pd(
    sheet_name="data",
    sheet_id=os.environ["SHEET_ID"],
    usecols=[
        "UID",
        "SUID",
        "NAME",
        "CHARACTER",
        "MAP",
        "PLACE",
        "PLAYERS",
        "DATE",
        "SEASON",
    ],
)

In [59]:
df

,UID,SUID,NAME,CHARACTER,MAP,PLACE,PLAYERS,DATE,SEASON
0,1,1,Cole,Toad,Sherbet Land,4,4,2021-09-20,0
1,1,1,Connor,Yoshi,Sherbet Land,2,4,2021-09-20,0
2,1,1,Cooper,Peach,Sherbet Land,1,4,2021-09-20,0
3,1,1,Triston,Bowser,Sherbet Land,3,4,2021-09-20,0
4,2,1,Cole,Toad,Kalimari Desert,4,4,2021-09-20,0
...,...,...,...,...,...,...,...,...,...
21641,6288,735,Regan,Yoshi,Moo Moo Farm,3,4,2024-07-15,11
21642,6288,735,Konnor,Bowser,Moo Moo Farm,4,4,2024-07-15,11
21643,6289,735,Regan,Toad,Bowser's Castle,1,3,2024-07-15,11
21644,6289,735,Cooper,D.K.,Bowser's Castle,2,3,2024-07-15,11


In [60]:
rated_seasons = []

for season in alive_it(
    it=df["SEASON"].unique(), total=df["SEASON"].unique().shape[0], theme="classic"
):

    tdf = (
        df[(df["PLAYERS"] == 4) & (df["SEASON"] == 11)]
        .drop(columns=["SUID", "CHARACTER", "PLAYERS", "SEASON", "MAP"])
        .sort_values(by=["UID", "PLACE"])
        .copy()
        .reset_index(drop=True)
    )

    pdf = (
        tdf.pivot(index="UID", columns="PLACE", values="NAME")
        .reset_index()
        .rename(columns={1: "1st", 2: "2nd", 3: "3rd", 4: "4th"})
        .merge(tdf[["UID", "DATE"]], on="UID", how="inner", validate="1:m")
        .groupby("UID")
        .first()
        .set_index("DATE")
        .reset_index()
        .rename(columns={"DATE": "date"})
    )

    exp_elo = MultiElo(score_function_base=1.5)
    tracker = Tracker(elo_rater=exp_elo)
    tracker.process_data(pdf)

    rdf = tracker.get_current_ratings()

    rdf["SEASON"] = season

    rated_seasons.append(rdf)

[========================================] 12/12 [100%] in 1.6s (7.46/s) 


In [61]:
rated_seasons[-1]

,rank,player_id,n_games,rating,SEASON
0,1,Cooper,142,1431.138670,11
1,2,Regan,58,1212.174791,11
2,3,Cole,33,1153.182061,11
3,4,Matt,68,1138.867777,11
4,5,Blake,29,1085.996876,11
5,6,Konnor,21,1075.849717,11
6,7,Luke,49,1066.633592,11
7,8,Chandler,11,1045.096382,11
8,9,Connor,24,1016.657037,11
9,10,Sam,2,978.537973,11


In [64]:
pd.concat(rated_seasons).sort_values(by=["SEASON", "rank"]).reset_index(drop=True)

,rank,player_id,n_games,rating,SEASON
0,1,Cooper,142,1431.138670,0
1,2,Regan,58,1212.174791,0
2,3,Cole,33,1153.182061,0
3,4,Matt,68,1138.867777,0
4,5,Blake,29,1085.996876,0
...,...,...,...,...,...
235,16,Randy,10,876.515824,11
236,17,Colton,21,872.667841,11
237,18,Domingo,26,840.244082,11
238,19,Antonio,15,839.396319,11


In [66]:
tdf

,UID,NAME,PLACE,DATE
0,1,Cooper,1,2021-09-20
1,1,Connor,2,2021-09-20
2,1,Triston,3,2021-09-20
3,1,Cole,4,2021-09-20
4,2,Cooper,1,2021-09-20
...,...,...,...,...
21641,6288,Regan,3,2024-07-15
21642,6288,Konnor,4,2024-07-15
21643,6289,Regan,1,2024-07-15
21644,6289,Cooper,2,2024-07-15


In [67]:
df

,UID,SUID,NAME,CHARACTER,MAP,PLACE,PLAYERS,DATE,SEASON
0,1,1,Cole,Toad,Sherbet Land,4,4,2021-09-20,0
1,1,1,Connor,Yoshi,Sherbet Land,2,4,2021-09-20,0
2,1,1,Cooper,Peach,Sherbet Land,1,4,2021-09-20,0
3,1,1,Triston,Bowser,Sherbet Land,3,4,2021-09-20,0
4,2,1,Cole,Toad,Kalimari Desert,4,4,2021-09-20,0
...,...,...,...,...,...,...,...,...,...
21641,6288,735,Regan,Yoshi,Moo Moo Farm,3,4,2024-07-15,11
21642,6288,735,Konnor,Bowser,Moo Moo Farm,4,4,2024-07-15,11
21643,6289,735,Regan,Toad,Bowser's Castle,1,3,2024-07-15,11
21644,6289,735,Cooper,D.K.,Bowser's Castle,2,3,2024-07-15,11


In [79]:
tdf["UID"].unique().shape[0]

6289

In [85]:
msk = (df["UID"].value_counts().reset_index()["count"] > 4 ) | (df["UID"].value_counts().reset_index()["count"] < 2)

df[df["UID"].isin(df["UID"].value_counts().index[msk])]

,UID,SUID,NAME,CHARACTER,MAP,PLACE,PLAYERS,DATE,SEASON
1123,313,25,Regan,Yoshi,Mario Raceway,1,4,2021-11-11,0
1124,313,25,Matt,Bowser,Mario Raceway,2,4,2021-11-11,0
1125,313,25,Blake,Peach,Mario Raceway,3,4,2021-11-11,0
1126,313,25,Connor,Toad,Mario Raceway,4,4,2021-11-11,0
1127,313,25,Matt,Yoshi,Wario Stadium,1,4,2021-11-11,0
...,...,...,...,...,...,...,...,...,...
20005,5821,696,Blake,Yoshi,Koopa Troopa Beach,3,4,2024-06-06,10
20006,5821,696,Graber,Peach,Koopa Troopa Beach,4,4,2024-06-06,10
20007,5821,696,Cooper,Wario,Bowser's Castle,1,3,2024-06-06,10
20008,5821,696,Blake,Toad,Bowser's Castle,2,3,2024-06-06,10


In [65]:
tdf = (
    df.drop(columns=["SUID", "CHARACTER", "PLAYERS", "SEASON", "MAP"])
    .sort_values(by=["UID", "PLACE"])
    .copy()
    .reset_index(drop=True)
)

pdf = (
    tdf.pivot(index="UID", columns="PLACE", values="NAME")
    .reset_index()
    .rename(columns={1: "1st", 2: "2nd", 3: "3rd", 4: "4th"})
    .merge(tdf[["UID", "DATE"]], on="UID", how="inner", validate="1:m")
    .groupby("UID")
    .first()
    .set_index("DATE")
    .reset_index()
    .rename(columns={"DATE": "date"})
)

exp_elo = MultiElo(score_function_base=1.5)
tracker = Tracker(elo_rater=exp_elo)
tracker.process_data(pdf)

rdf = tracker.get_current_ratings()

ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
# import altair as alt

# df = tracker.get_history_df()
# alt.Chart(df).mark_line(point=True).encode(
#     x="date:T",
#     y=alt.Y("rating:Q", scale=alt.Scale(zero=False)),
#     color="player_id:N",
# )